In [1]:
import numpy as np
import pandas as pd
import re
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
nltk.download("stopwords")
nltk.download("punkt")


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
feature=pd.read_csv("/kaggle/input/tweet-real-fake-account/Features_For_Traditional_ML_Techniques.csv",nrows=500)
truth_seek=pd.read_csv("/kaggle/input/tweet-real-fake-account/Truth_Seeker_Model_Dataset.csv")
truth_seek_excel=pd.read_excel("/kaggle/input/tweet-real-fake-account/Truth_Seeker_Model_Dataset_With_TimeStamps 1.xlsx")

In [3]:
feature.sample()

,Unnamed: 0,majority_target,statement,BinaryNumTarget,tweet,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,determiners,conjunctions,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq
307,307,True,End of eviction moratorium means millions of A...,1.0,Americans in need of the eviction moratorium e...,1231.0,1131.0,4670.0,4527.0,2.0,...,0,0,3,1,1,0,8,0,4,39


In [4]:
# feature["embeddings"]

In [5]:
feature.select_dtypes(include=["object", "category"]).sample()


,statement,tweet,embeddings
240,End of eviction moratorium means millions of A...,The end of the Eviction Moratorium affects a l...,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [6]:
truth_seek.sample()

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer
125413,125413,Jessica Calefati,"The Trump administration added over 209,000 jo...",False,0.0,"Trump administration, added, jobs",@n_thrope @smelluminati @WeDoNotLearn73 Prior ...,Agree,Agree


In [7]:
truth_seek_excel.sample()

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,timestamp
2749,2749,Chris Nichols,San Francisco had twice as many drug overdose ...,1.0,1.0,"San Francisco, drug overdose",@ksorbs Yet in Nancy Pelosi district. San Fra...,NO MAJORITY,Agree,Mon Dec 21 21:13:22 +0000 2020


In [8]:
def text_preprocess(df,columns):
#     processed_col=[]
    for col in columns:
        df[col]=df[col].str.lower()
        data_re=df[col].apply(lambda x:re.sub(r"\W+"," ",str(x)))
        data_tokenize=data_re.apply(word_tokenize)
        tokens=data_tokenize.apply(lambda x:[word for word in x if word not in stopwords.words("english")])
        df[col]=(tokens.apply(lambda x:" ".join(x)))
    return df
col=["statement","tweet","embeddings"]
feature=text_preprocess(feature,col)

In [9]:
feature["embeddings"]

0      0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1      0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2      0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
3      0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
4      0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
                     ...                
495    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
496    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
497    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
498    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
499    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Name: embeddings, Length: 500, dtype: object

In [10]:
def text_vectorizer(df,columns):
    combine_vector=df[columns].apply(lambda x:" ".join(x),axis=1)
    vectorizer=TfidfVectorizer()
    vectors=vectorizer.fit_transform(combine_vector)
    vector=pd.DataFrame(vectors.toarray(),columns=vectorizer.get_feature_names_out())
    return vector

In [11]:
vect=text_vectorizer(feature,["statement","tweet"])

In [12]:
vect

,000,03,05,10,100,1000s,100m,100s,10am,10pm,...,york,yorkers,youngbiafra,youre,yr,ytd,yvette,zaleskiluke,zero,zone
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.116787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
496,0.127507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
497,0.137725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.181579,0.0,0.0,0.0,0.0,0.0,0.0
498,0.101941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
feature["majority_target"]=feature["majority_target"].astype(int)

In [14]:
# feature["embeddings"]

In [15]:
concaty=pd.concat([feature,vect],axis=1).drop(columns=col)

In [16]:
concaty

,Unnamed: 0,majority_target,BinaryNumTarget,followers_count,friends_count,favourites_count,statuses_count,listed_count,following,BotScore,...,york,yorkers,youngbiafra,youre,yr,ytd,yvette,zaleskiluke,zero,zone
0,0,1,1.0,4262.0,3619.0,34945.0,16423.0,44.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1.0,1393.0,1621.0,31436.0,37184.0,64.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,1.0,9.0,84.0,219.0,1184.0,0.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1,1.0,4262.0,3619.0,34945.0,16423.0,44.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1,1.0,70.0,166.0,15282.0,2194.0,0.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,1,1.0,59.0,384.0,1432.0,5483.0,0.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
496,496,1,1.0,87.0,201.0,18059.0,11389.0,0.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
497,497,1,1.0,406.0,1241.0,11417.0,14999.0,14.0,0.0,0.03,...,0.0,0.0,0.0,0.181579,0.0,0.0,0.0,0.0,0.0,0.0
498,498,1,1.0,48.0,183.0,4874.0,7863.0,6.0,0.0,0.03,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
concaty.iloc[:,0:60].dtypes

Unnamed: 0                  int64
majority_target             int64
BinaryNumTarget           float64
followers_count           float64
friends_count             float64
favourites_count          float64
statuses_count            float64
listed_count              float64
following                 float64
BotScore                  float64
BotScoreBinary            float64
cred                      float64
normalize_influence       float64
mentions                  float64
quotes                    float64
replies                   float64
retweets                  float64
favourites                float64
hashtags                  float64
URLs                      float64
unique_count                int64
total_count                 int64
ORG_percentage            float64
NORP_percentage           float64
GPE_percentage            float64
PERSON_percentage         float64
MONEY_percentage          float64
DATE_percentage           float64
CARDINAL_percentage       float64
PERCENT_percen

In [18]:
X=concaty.drop(columns=["Unnamed: 0","majority_target","BinaryNumTarget"])
Y=concaty["majority_target"]

In [19]:
Y.unique()

array([1, 0])

In [20]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [21]:
X_train.sample()

,followers_count,friends_count,favourites_count,statuses_count,listed_count,following,BotScore,BotScoreBinary,cred,normalize_influence,...,york,yorkers,youngbiafra,youre,yr,ytd,yvette,zaleskiluke,zero,zone
64,4262.0,3619.0,34945.0,16423.0,44.0,0.0,0.03,0.0,0.540794,0.104602,...,0.0,0.0,0.0,0.0,0.0,0.201325,0.0,0.0,0.0,0.0


In [22]:
models=[RandomForestClassifier(),GradientBoostingClassifier(),DecisionTreeClassifier(),AdaBoostClassifier(),BaggingClassifier(),LogisticRegression(),SVC(),KNeighborsClassifier(),BernoulliNB(),MLPClassifier()]
for i in models:
    model=i
    model.fit(X_train,Y_train)
    print(f" The accuracy of {i} is",model.score(X_test,Y_test))

 The accuracy of RandomForestClassifier() is 0.93
 The accuracy of GradientBoostingClassifier() is 0.89
 The accuracy of DecisionTreeClassifier() is 0.87
 The accuracy of AdaBoostClassifier() is 0.87
 The accuracy of BaggingClassifier() is 0.93


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


 The accuracy of LogisticRegression() is 0.93
 The accuracy of SVC() is 0.93
 The accuracy of KNeighborsClassifier() is 0.92
 The accuracy of BernoulliNB() is 0.93
 The accuracy of MLPClassifier() is 0.93


# # RNN

In [23]:
X_train_reshape=X_train.values
X_train_reshape=np.repeat(X_train_reshape[:,np.newaxis,:],5,axis=1)
X_train_reshape.shape
X_test_reshape=X_test.values
X_test_reshape=np.repeat(X_test_reshape[:,np.newaxis,:],5,axis=1)
X_test_reshape.shape

(100, 5, 3423)

In [24]:
from keras.layers import Dense,Flatten,LSTM,SimpleRNN,GRU,BatchNormalization,Dropout,Input,LeakyReLU,Layer
from keras.models import Model,Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2
import tensorflow as tf
from keras.callbacks import EarlyStopping

In [25]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
verbose=1)

In [26]:
inputs=Input(shape=(5,3423))
S=SimpleRNN(32,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=LSTM(32,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=LeakyReLU(0.08)(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=LSTM(64,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=LeakyReLU(0.08)(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=LSTM(128,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=LeakyReLU(0.08)(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=GRU(256,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=LeakyReLU(0.08)(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=GRU(556,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=LeakyReLU(0.08)(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=Flatten()(S)
S=Dense(32,activation="relu")(S)
outputs=Dense(1,activation="sigmoid")(S)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5, 3423)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 5, 32)          │       110,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 5, 32)          │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 5, 32)          │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5, 32)          │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 5, 64)          │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 5, 128)         │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 5, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 5, 256)         │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 5, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 5, 556)         │     1,357,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 5, 556)         │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,990,057 (7.59 MB)

 Trainable params: 1,987,921 (7.58 MB)

 Non-trainable params: 2,136 (8.34 KB)

In [27]:
model.fit(X_train_reshape,Y_train,epochs=50,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 195ms/step - accuracy: 0.7596 - loss: 66.0337 - val_accuracy: 0.9750 - val_loss: 60.7589
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9443 - loss: 58.8062 - val_accuracy: 0.9750 - val_loss: 54.2299
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.9447 - loss: 52.5255 - val_accuracy: 0.9750 - val_loss: 48.5606
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.9297 - loss: 47.0656 - val_accuracy: 0.9750 - val_loss: 43.6525
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.9315 - loss: 42.1965 - val_accuracy: 0.9750 - val_loss: 39.2899
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9393 - loss: 38.0087 - val_accuracy: 0.9750 - val_loss: 35.4386
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9295 - loss: 34.3302 - val_accuracy: 0.9750 - val_loss: 32.0847
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.9335 - loss: 31.0637 - val_acc

In [28]:
loss,accuracy=model.evaluate(X_test_reshape,Y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9366 - loss: 1.9757


# # QuantumRNN

In [29]:
import cirq

In [30]:
!pip install cirq

In [31]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(len(data))]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        circuit.append(cirq.rx(value*np.pi).on(qubit[i]))
    for i in range(len(data)-1):
        circuit.append(cirq.CNOT(qubit[i],qubit[i+1]))
        circuit.append(cirq.measure(qubit[i],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for inputs_data in data:
            circuit,qubit=quantum_circuit(inputs_data)
            simulator=cirq.Simulator()
            result=simulator.run(circuit)
            for i in range(len(inputs_data)):
                results.append(measure0=result.measurements[f"qubit_{i}"].flatten()[0])
    return np.array(results,dtypes=np.float32)

class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        inputs_flat=tf.reshape(inputs,[-1,inputs.shape[-1]])
        output=tf.numpy_function(run_quantum_circuit,[inputs_flat],tf.float32)
        output.set_shape(inputs.shape)
        return output
modelq=Sequential([
    QuantumLayer(),
#     GRU(56,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
#     LeakyReLU(0.08),
#     Dropout(0.5),
#     BatchNormalization(),
    Flatten(),
    Dense(1,activation="sigmoid")
])
modelq.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
modelq.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ quantum_layer (QuantumLayer)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
X_train_reshape.shape

(400, 5, 3423)

In [ ]:
modelq.fit(X_train_reshape,Y_train,epochs=50,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
